# Necessary modules and environment params

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

In [3]:
from src.BaseModel import BaseModel
from src.Evaluator import Evaluator
from src.ALS_MF import ALS_MF
from src.data_pipeline import *
from src.utils import get_spark

In [4]:
spark = get_spark()

# Load whole data from the text file

In [5]:
whole_data = load_data(spark)

/home/henryhuang/Desktop/Course/Personalization/code/Personalization_Proj1/data/ml-1m/ratings.dat


# Split train and test set

In [6]:
split_params = {
    "partition_by": "user",
    "rank_method": "random",
} # default setting

train, test = train_test_split(whole_data, ratio_range=(0,0.2), **split_params)
# same to train_test_split(whole_data)

Using split of range (0, 0.2), test set contains 202451 of 1000209 records in total.


# Construct Model

In [7]:
als_params = {
    "rank": 10,
    "maxIter": 15,
    "implicitPrefs": False,
    "alpha": 0.0,
    "regParam": 0.05,
    "coldStartStrategy": "drop",
    "nonnegative": False
}

In [8]:
als = ALS_MF(als_params)

# Cross validation

In [9]:
cross_validation(train, als, top_k=10, num_candidates=1000, split_params=split_params, k_fold=4)

Using split of range [0.0, 0.25], test set contains 201248 of 797758 records in total.
Using split of range [0.25, 0.5], test set contains 200070 of 797758 records in total.
Using split of range [0.5, 0.75], test set contains 200070 of 797758 records in total.
Using split of range [0.75, 1.0], test set contains 201248 of 797758 records in total.


defaultdict(list,
            {'ndcg': [0.01311504925552225,
              0.016180484322696663,
              0.019742666350438642,
              0.015794973090892027],
             'precision': [0.01582781456953642,
              0.018807947019867554,
              0.022798013245033107,
              0.018890728476821193]})

# Finally train model using all training data, and evaluate on test data

In [10]:
best_params = {
    "rank": 64,
    "maxIter": 15,
    "implicitPrefs": False,
    "alpha": 0.3,
    "regParam": 0.05,
    "coldStartStrategy": "drop",
    "nonnegative": False
}
als_final = ALS_MF(best_params)

evaluate_model(train, test, als_final, top_k=10, num_candidates=50)

defaultdict(list,
            {'ndcg': [0.055878182548301396],
             'precision': [0.05251655629139074]})